* 출처 : https://keras.io/ko/applications/


In [ ]:
%%bash
[ ! -f flower_photos_300x200_small_train_test2.zip ]&& wget https://raw.githubusercontent.com/Finfra/AI_Vision/master/data/flower_photos_300x200_small_train_test2.zip

rm -rf __MACOSX
rm -rf flowers
unzip -q flower_photos_300x200_small_train_test2.zip
mv flower_photos_300x200_small_train_test2 flowers

cd flowers


find .|grep .DS_Store|xargs rm -f
find .|head -n 10


In [2]:
from os import listdir
from os.path import isfile, join, splitext
import cv2
def getFolder(thePath,isFile=True):
    return [f for f in listdir(thePath) if isFile == isfile(join(thePath, f)) ]

def convert(thePath,to_w,to_h):
    ext=splitext(".jpg")[0]
    if ext in ('.jpg','.png'):
      img = cv2.imread(thePath)
      if (to_h,to_w,3) != img.shape :
        img = cv2.resize(img,(to_w,to_h))
        print(img.shape[0],img.shape[1],'to',to_w,to_h , thePath , )
        cv2.imwrite(thePath,img)
      # else:
      #   print(thePath,"is not changed (same)")

def convertAll(tPath,to_w,to_h):
  for folder in getFolder(tPath,False):
    print('-------------------')
    print(join(tPath,folder))
    convertAll(join(tPath,folder),to_w,to_h)
  for files in getFolder(tPath,True):
      convert(join(tPath,files),to_w,to_h)


tPath='/content/flowers/'


In [11]:
!apt-get install imagemagick


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono ghostscript gsfonts
  imagemagick-6-common imagemagick-6.q16 libcupsfilters1 libcupsimage2
  libdjvulibre-text libdjvulibre21 libgs9 libgs9-common libijs-0.35
  libjbig2dec0 liblqr-1-0 libmagickcore-6.q16-3 libmagickcore-6.q16-3-extra
  libmagickwand-6.q16-3 libnetpbm10 libwmf0.2-7 netpbm poppler-data
Suggested packages:
  fonts-noto ghostscript-x imagemagick-doc autotrace cups-bsd | lpr | lprng
  enscript gimp gnuplot grads hp2xx html2ps libwmf-bin mplayer povray radiance
  sane-utils texlive-base-bin transfig ufraw-batch inkscape libjxr-tools
  libwmf0.2-7-gtk poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono g

In [10]:
!identify /content/flowers/test/daisy/14221836990_90374e6b34.jpg

/bin/bash: convert: command not found


In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

w=224
h=224
color=3

convertAll(tPath,w,h)



# load and iterate training dataset
datagen = ImageDataGenerator()
train_data = datagen.flow_from_directory(directory='flowers/train/', class_mode='categorical', batch_size=64,target_size=(h, w))
# load and iterate test dataset
test_data = datagen.flow_from_directory(directory='flowers/test/',  class_mode='categorical', batch_size=64,target_size=(h,w))



-------------------
/content/flowers/train
-------------------
/content/flowers/train/tulip
224 224 to 224 224 /content/flowers/train/tulip/130685245_dcdd23836f_m.jpg
224 224 to 224 224 /content/flowers/train/tulip/142235017_07816937c6.jpg
224 224 to 224 224 /content/flowers/train/tulip/6958243974_8851425ddb_n.jpg
224 224 to 224 224 /content/flowers/train/tulip/2229804138_db9cba3443_n.jpg
224 224 to 224 224 /content/flowers/train/tulip/3601085193_de1195d3d7_n.jpg
224 224 to 224 224 /content/flowers/train/tulip/14674071872_2df55466d5_m.jpg
224 224 to 224 224 /content/flowers/train/tulip/16506668270_b823935dc3.jpg
224 224 to 224 224 /content/flowers/train/tulip/4571353297_5634177744_n.jpg
224 224 to 224 224 /content/flowers/train/tulip/2481015475_b71a12917d.jpg
224 224 to 224 224 /content/flowers/train/tulip/18245124970_e68fd3f3c3.jpg
224 224 to 224 224 /content/flowers/train/tulip/8687675254_c93f50d8b0_m.jpg
224 224 to 224 224 /content/flowers/train/tulip/8713387500_6a9138b41b_n.jpg
224

In [13]:
for image_batch, label_batch in train_data:
  print("Image batch shape: ", image_batch.shape)
  print("Label batch shape: ", label_batch.shape)
  label_set_count=label_batch.shape[1]
  break


Image batch shape:  (64, 224, 224, 3)
Label batch shape:  (64, 2)


In [14]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

# 선행학습된 기준모델을 만듭니다
base_model = InceptionV3(weights='imagenet', include_top=False)

# 글로벌 공간 평균값 풀링 레이어를 더합니다
x = base_model.output
x = GlobalAveragePooling2D()(x)
# 완전 연결 레이어를 더합니다
x = Dense(1024, activation='relu')(x)
# 로지스틱 레이어를 더합니다 -- 200가지 클래스가 있다고 가정합니다
predictions = Dense(label_set_count, activation='softmax')(x)

# 다음은 학습할 모델입니다
model = Model(inputs=base_model.input, outputs=predictions)

# 첫째로: (난수로 초기값이 설정된) 가장 상위 레이어들만 학습시킵니다
# 다시 말해서 모든 InceptionV3 콘볼루션 레이어를 고정합니다
for layer in base_model.layers:
    layer.trainable = False

# 모델을 컴파일합니다 (*꼭* 레이어를 학습불가 상태로 세팅하고난 *후*에 컴파일합니다)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])



# 모델을 새로운 데이터에 대해 몇 세대간 학습합니다
model.fit_generator(train_data, 
                    validation_data=test_data, 
                    validation_steps=8,
                    epochs=50
)

87924736/87910968 [==============================] - 1s 0us/step


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/50
3/3 [==============================] - 20s 2s/step - loss: 327.6520 - accuracy: 0.4788 - val_loss: 54.4965 - val_accuracy: 0.4857
Epoch 2/50
3/3 [==============================] - 1s 189ms/step - loss: 35.5246 - accuracy: 0.5394
Epoch 3/50
3/3 [==============================] - 1s 140ms/step - loss: 22.3948 - accuracy: 0.5394
Epoch 4/50
3/3 [==============================] - 1s 139ms/step - loss: 21.1054 - accuracy: 0.5394
Epoch 5/50
3/3 [==============================] - 1s 143ms/step - loss: 12.6726 - accuracy: 0.5515
Epoch 6/50
3/3 [==============================] - 1s 185ms/step - loss: 9.9751 - accuracy: 0.5697
Epoch 7/50
3/3 [==============================] - 1s 159ms/step - loss: 13.3273 - accuracy: 0.5152
Epoch 8/50
3/3 [==============================] - 1s 141ms/step - loss: 6.5070 - accuracy: 0.5879
Epoch 9/50
3/3 [==============================] - 1s 188ms/step - loss: 7.4468 - accuracy: 0.6182
Epoch 10/50
3/3 [==============================] - 1s 190ms/step - los

In [ ]:
score = model.evaluate_generator(test_data)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Please use Model.evaluate, which supports generators.
Test loss: 1.8776918649673462
Test accuracy: 0.6571428775787354


In [ ]:
# 이 시점에서 상위 레이어들은 충분히 학습이 되었기에,
# inception V3의 콘볼루션 레이어에 대한 파인튜닝을 시작합니다 
# 가장 밑 N개의 레이어를 고정하고 나머지 상위 레이어를 학습시킵니다

# 레이어 이름과 레이어 인덱스를 시각화하여
# 얼마나 많은 레이어를 고정시켜야 하는지 확인합니다:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)



0 input_1
1 conv2d
2 batch_normalization
3 activation
4 conv2d_1
5 batch_normalization_1
6 activation_1
7 conv2d_2
8 batch_normalization_2
9 activation_2
10 max_pooling2d
11 conv2d_3
12 batch_normalization_3
13 activation_3
14 conv2d_4
15 batch_normalization_4
16 activation_4
17 max_pooling2d_1
18 conv2d_8
19 batch_normalization_8
20 activation_8
21 conv2d_6
22 conv2d_9
23 batch_normalization_6
24 batch_normalization_9
25 activation_6
26 activation_9
27 average_pooling2d
28 conv2d_5
29 conv2d_7
30 conv2d_10
31 conv2d_11
32 batch_normalization_5
33 batch_normalization_7
34 batch_normalization_10
35 batch_normalization_11
36 activation_5
37 activation_7
38 activation_10
39 activation_11
40 mixed0
41 conv2d_15
42 batch_normalization_15
43 activation_15
44 conv2d_13
45 conv2d_16
46 batch_normalization_13
47 batch_normalization_16
48 activation_13
49 activation_16
50 average_pooling2d_1
51 conv2d_12
52 conv2d_14
53 conv2d_17
54 conv2d_18
55 batch_normalization_12
56 batch_normalization_14
5

In [16]:
# 가장 상위 2개의 inception 블록을 학습하기로 고릅니다,
# 다시 말하면 첫 249개의 레이어는 고정시키고 나머지는 고정하지 않습니다:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True



In [18]:
# 이러한 수정사항이 효과를 내려면 모델을 다시 컴파일해야 합니다
# 낮은 학습 속도로 세팅된 SGD를 사용합니다
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy',metrics=['accuracy'])

# 다시 한 번 모델을 학습시킵니다
# (이번엔 상위 2개의 inception 블록을 상위의 밀집 레이어들과 함께 파인튜닝합니다)
model.fit_generator(train_data, 
                    validation_data=test_data, 
                    validation_steps=8,
                    epochs=100
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/100
3/3 [==============================] - 11s 1s/step - loss: 0.8890 - accuracy: 0.4970 - val_loss: 0.9199 - val_accuracy: 0.6000
Epoch 2/100
3/3 [==============================] - 1s 159ms/step - loss: 0.8799 - accuracy: 0.4970
Epoch 3/100
3/3 [==============================] - 1s 194ms/step - loss: 0.8540 - accuracy: 0.4970
Epoch 4/100
3/3 [==============================] - 1s 147ms/step - loss: 0.8332 - accuracy: 0.4970
Epoch 5/100
3/3 [==============================] - 1s 141ms/step - loss: 0.8026 - accuracy: 0.4970
Epoch 6/100
3/3 [==============================] - 1s 154ms/step - loss: 0.7718 - accuracy: 0.4970
Epoch 7/100
3/3 [==============================] - 1s 193ms/step - loss: 0.7478 - accuracy: 0.4970
Epoch 8/100
3/3 [==============================] - 1s 162ms/step - loss: 0.7364 - accuracy: 0.4970
Epoch 9/100
3/3 [==============================] - 1s 150ms/step - loss: 0.7128 - accuracy: 0.4970
Epoch 10/100
3/3 [==============================] - 1s 190ms/step - l

In [19]:
score = model.evaluate_generator(test_data)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


Test loss: 0.5066385269165039
Test accuracy: 0.7428571581840515
